In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import glob
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier#1
from sklearn.tree import DecisionTreeClassifier#2
from sklearn.ensemble import RandomForestClassifier#3
from sklearn.linear_model import LogisticRegression#4
from sklearn.svm import SVC#5
from xgboost import XGBClassifier#6
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support, classification_report
from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing
from PIL import Image
import skimage
from PIL import Image
import IPython.display as display
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
def image_to_feature_vector(image, size=(150, 150)):
        
     # resize the image to a fixed size, then flatten the image into
     # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(32, 32, 32)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	hist = cv2.calcHist([rgb], [0, 1, 2], None, bins,[0, 256, 0, 256, 0, 256])

	# handle normalizing the histogram if we are using OpenCV 2.4.X
	hist = cv2.normalize(hist, hist)
	flatten_feature = hist.flatten()
	# return the flattened histogram as the feature vector
	return flatten_feature

In [4]:
#dataset1


dir1 = '/content/drive/MyDrive/تسک 03_دیتاهای اب مروارید/ALL_03/ALL/ALL_new'
#dir1 = 'E:/Heart/Universal/final'
def fetch_cataract(dir):
    raw_image = []
    health_state =[]
    features = []
    c=0

    for filename in os.listdir(dir1):
        category = filename.split('_')[0]
        if category == 'cat':
                health_state.append(1)
        else:
                health_state.append(0)
        image =cv2.imread(os.path.join(dir1,filename))
        flatten_feature = extract_color_histogram(image, bins=(32, 32, 32))
        pixel = image_to_feature_vector(image, size=(150, 150))
        raw_image.append(pixel)
        features.append(flatten_feature)
    raw_image = np.array(raw_image)
    health_state = np.array(health_state)
    features = np.array(features)
    #df=pd.DataFrame({ 'pixel_image':os.listdir(dir1), 'features':features, 'HEALTH':health_state})
    return raw_image, health_state, features 
raw_image, health_state, features = fetch_cataract(dir1)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
(trainX, testX, trainY, testY ) = train_test_split(features, health_state, test_size= 0.3, random_state=20,shuffle=True)

In [6]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(112, 32768)
(112,)
(49, 32768)
(49,)


In [7]:
# k-NN
model_knn = KNeighborsClassifier(n_neighbors=10)
model_knn.fit(trainX, trainY)
acc = model_knn.score(testX, testY)
y_pred_rgb_knn = model_knn.predict(testX)
print("y_pred_RF=",y_pred_rgb_knn)
print("acc= ",acc)
print("MAE= ",mean_absolute_error(y_pred_rgb_knn,testY))  #MAE
print("MSE= ",mean_squared_error(y_pred_rgb_knn,testY))  #MSE

y_pred_RF= [1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0
 1 1 1 0 1 1 0 0 0 1 0 0]
acc=  0.7755102040816326
MAE=  0.22448979591836735
MSE=  0.22448979591836735


In [8]:
#Decision Tree

model_DT=DecisionTreeClassifier()
model_DT.fit(trainX, trainY)
acc = model_DT.score(testX, testY)
y_pred_DT = model_DT.predict(testX)
print("y_pred_DT=",y_pred_DT)
print("acc= ",acc)
print("MAE= ",mean_absolute_error(y_pred_DT,testY))  #MAE
print("MSE= ",mean_squared_error(y_pred_DT,testY))  #MSE

y_pred_DT= [1 1 1 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0
 1 0 0 0 1 1 1 0 0 1 0 0]
acc=  0.8979591836734694
MAE=  0.10204081632653061
MSE=  0.10204081632653061


In [9]:
#Random Forest

model_RF = RandomForestClassifier(n_estimators =12, max_depth =10, random_state=25) 
model_RF.fit(trainX, trainY) 
y_pred_RF = model_RF.predict(testX)
acc = model_RF.score(testX, testY)
print("y_pred_RF=",y_pred_rgb_knn)
print("acc= ",acc)
print("MAE= ",mean_absolute_error(y_pred_RF,testY))  #MAE
print("MSE= ",mean_squared_error(y_pred_RF,testY))  #MSE

y_pred_RF= [1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0
 1 1 1 0 1 1 0 0 0 1 0 0]
acc=  0.9387755102040817
MAE=  0.061224489795918366
MSE=  0.061224489795918366


In [10]:
#Logistic Regression

model_LR = LogisticRegression(penalty ='l2',C=1,random_state=25) 
model_LR.fit(trainX, trainY)
y_pred_LR = model_LR.predict(testX)
acc = model_LR.score(testX, testY)
print("y_pred_RF=",y_pred_LR)
print("acc= ",acc)
print("MAE= ",mean_absolute_error(y_pred_LR,testY))  #MAE
print("MSE= ",mean_squared_error(y_pred_LR,testY))  #MSE

y_pred_RF= [1 1 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0
 1 0 1 0 1 1 0 0 0 1 0 0]
acc=  0.8163265306122449
MAE=  0.1836734693877551
MSE=  0.1836734693877551


In [11]:
#SVC

model_SVC = SVC(max_iter=1000,class_weight='balanced')
model_SVC.fit(trainX, trainY)
acc = model_SVC.score(testX, testY)
y_pred_SVC = model_SVC.predict(testX)
print("y_pred_SVC=",y_pred_LR)
print("acc= ",acc)
print("MAE= ",mean_absolute_error(y_pred_SVC,testY))  #MAE
print("MSE= ",mean_squared_error(y_pred_SVC,testY))  #MSE

y_pred_SVC= [1 1 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0
 1 0 1 0 1 1 0 0 0 1 0 0]
acc=  0.8571428571428571
MAE=  0.14285714285714285
MSE=  0.14285714285714285


In [12]:
#XGBoost

model_xgb=XGBClassifier()
model_xgb.fit(trainX,trainY)
y_pred_xgb = model_xgb.predict(testX)
print("y_pred_xgb=",y_pred_xgb)
print("acc= ",acc)
print("MAE= ",mean_absolute_error(y_pred_xgb,testY))  #MAE
print("MSE= ",mean_squared_error(y_pred_xgb,testY))  #MSE

y_pred_xgb= [1 1 1 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0
 1 0 1 0 1 1 0 0 0 1 0 0]
acc=  0.8571428571428571
MAE=  0.10204081632653061
MSE=  0.10204081632653061


In [16]:
#cross validation 

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

scores_RF=cross_val_score(model_RF,trainX, trainY,cv=5)
K_fold_CV_Score_RF = round(scores_RF.mean()*100)
print(scores_RF)
print("mean of K_fold_CV_Score_RF is =",K_fold_CV_Score_RF)
#cv= 10 -> mean of K_fold_CV_Score_RF=95
#cv= 5 -> mean of K_fold_CV_Score_RF=96

[0.91304348 1.         0.95454545 1.         0.90909091]
mean of K_fold_CV_Score_RF is = 96


In [17]:
scores_xgb=cross_val_score(model_xgb,trainX, trainY,cv=5)
K_fold_CV_Score_xgb = round(scores_xgb.mean()*100)
print(scores_xgb)
print("mean of K_fold_CV_Score_xgb is =",K_fold_CV_Score_xgb)
#cv= 10,5 -> mean of K_fold_CV_Score_xgb=96

[1.         0.95652174 1.         0.95454545 0.90909091]
mean of K_fold_CV_Score_xgb is = 96


In [20]:
scores_LR=cross_val_score(model_LR,trainX, trainY,cv=5)
K_fold_CV_Score_LR = round(scores_LR.mean()*100)
print(scores_LR)
print("mean of K_fold_CV_Score_LR is =",K_fold_CV_Score_LR)
#cv= 10 -> mean of K_fold_CV_Score_LR=87
#cv= 5 -> mean of K_fold_CV_Score_LR=85

[0.73913043 0.82608696 0.90909091 0.86363636 0.90909091]
mean of K_fold_CV_Score_LR is = 85


In [23]:
scores_SVC=cross_val_score(model_SVC,trainX, trainY,cv=5)
K_fold_CV_Score_SVC = round(scores_SVC.mean()*100)
print(scores_SVC)
print("mean of K_fold_CV_Score_SVC is =",K_fold_CV_Score_SVC)
#cv= 10,5 -> mean of K_fold_CV_Score_SVC=89

[0.82608696 0.86956522 0.95454545 0.90909091 0.90909091]
mean of K_fold_CV_Score_SVC is = 89


In [25]:
scores_DT=cross_val_score(model_DT,trainX, trainY,cv=5)
K_fold_CV_Score_DT = round(scores_DT.mean()*100)
print(scores_DT)
print("mean of K_fold_CV_Score_DT is =",K_fold_CV_Score_DT)
#cv= 10,5 -> mean of K_fold_CV_Score_DT=97

[1.         0.95652174 1.         0.95454545 0.95454545]
mean of K_fold_CV_Score_DT is = 97


In [28]:
scores_knn=cross_val_score(model_knn,trainX, trainY,cv=5)
K_fold_CV_Score_knn = round(scores_knn.mean()*100)
print(scores_knn)
print("mean of K_fold_CV_Score_knn is =",K_fold_CV_Score_knn)
#cv= 10 -> mean of K_fold_CV_Score_knn=84
#cv= 5 -> mean of K_fold_CV_Score_knn=85

[0.86956522 0.7826087  0.77272727 0.95454545 0.86363636]
mean of K_fold_CV_Score_knn is = 85
